In [1]:
%cd ..

/home/jbananafish/Desktop/Master/Thesis/code/gcnboost


In [2]:
from tqdm import tqdm
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, SAGEConv, to_hetero
import torch_geometric.transforms as T

from src.data.artgraph import ArtGraph

/home/jbananafish/anaconda3/envs/thesis-project/lib/python3.9/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  /opt/conda/conda-bld/pytorch_1631630797748/work/c10/cuda/CUDAFunctions.cpp:115.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
torch.manual_seed(1)
torch.cuda.manual_seed(1)

In [4]:
#base_data = ArtGraph("./ekg", preprocess='node2vec', features=True, type='ekg')
base_data = ArtGraph("data", preprocess='node2vec', transform=T.ToUndirected(), features=True, type='ekg')

## Some graph-level information

In [11]:
print(f"Number of artist classes: {base_data.num_classes['artist']}")
print(f"Number of style classes: {base_data.num_classes['style']}")
print(f"Number of genre classes: {base_data.num_classes['genre']}")
print(f"Number of input features: {base_data.num_features}")

Number of artist classes: 300
Number of style classes: 83
Number of genre classes: 50
Number of input features: 128


## Some node-level information

In [12]:
data = base_data[0]

In [13]:
print(data)

HeteroData(
  artwork={
    x=[61477, 128],
    y_artist=[61477],
    y_style=[61477],
    y_genre=[61477],
    train_mask=[61477],
    val_mask=[61477],
    test_mask=[61477]
  },
  artist={ x=[300, 128] },
  gallery={ x=[1090, 128] },
  city={ x=[665, 128] },
  country={ x=[64, 128] },
  style={ x=[83, 128] },
  period={ x=[53, 128] },
  genre={ x=[50, 128] },
  serie={ x=[610, 128] },
  auction={ x=[5, 128] },
  tag={ x=[5146, 128] },
  media={ x=[160, 128] },
  subject={ x=[2161, 128] },
  training_node={ x=[108, 128] },
  field={ x=[65, 128] },
  movement={ x=[121, 128] },
  people={ x=[48, 128] },
  (artist, influenced_rel, artist)={ edge_index=[2, 62] },
  (artist, subject_rel, subject)={ edge_index=[2, 3648] },
  (artist, training_rel, training_node)={ edge_index=[2, 130] },
  (artist, field_rel, field)={ edge_index=[2, 323] },
  (artist, movement_rel, movement)={ edge_index=[2, 286] },
  (artist, patrons_rel, people)={ edge_index=[2, 45] },
  (artist, teacher_rel, artist)={ ed

In [14]:
class GNN(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super(GNN, self).__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), out_channels)

    def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = F.dropout(x, 0.5)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

In [15]:
class MGNN(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels, metadata):
        super(MGNN, self).__init__()
        self.gnn_artist = GNN(hidden_channels, out_channels['artist'])
        self.gnn_artist = to_hetero(self.gnn_artist, metadata)

        self.gnn_style = GNN(hidden_channels, out_channels['style'])
        self.gnn_style = to_hetero(self.gnn_style, metadata)

        self.gnn_genre = GNN(hidden_channels, out_channels['genre'])
        self.gnn_genre = to_hetero(self.gnn_genre, metadata)

    def forward(self, x, edge_index):
        return [self.gnn_artist(x, edge_index), self.gnn_style(x, edge_index), self.gnn_genre(x, edge_index)]

In [16]:
class ArtGraphGCNBoost:

    map_labels = {
        0: 'artist',
        1: 'style',
        2: 'genre'
    }

    def __init__(self, model, data, optimizer):
        
        self.data = data
        self.artworks = data['artwork']
        self.y = torch.stack([data['artwork'].y_artist, data['artwork'].y_style, data['artwork'].y_genre])
        self.train_mask = self.artworks.train_mask
        self.val_mask = self.artworks.val_mask
        self.test_mask = self.artworks.test_mask

        self.model = model
        self.optimizer = optimizer

    def get_classes(self, label= 'artist', split='train'):
        pass 

    def get_accuracy(self, predicted, labels):
        return predicted.argmax(dim=1).eq(labels).sum()/predicted.shape[0]

    def get_accuracies(self, predicted, labels, mask):
        accuracies = [] 
        for id, _ in self.map_labels.items():
            accuracies.append(self.get_accuracy(predicted[id]['artwork'][mask], labels[id][mask]))
        return accuracies

    def get_loss(self, predicted, labels):
        return F.nll_loss(predicted, labels.type(torch.LongTensor))

    def get_losses(self, predicted, labels, mask):
        losses = []
        for id, _ in self.map_labels.items():
            losses.append(self.get_loss(predicted[id]['artwork'][mask], labels[id][mask]))
        return losses

    def train(self, epochs):
        for epoch in tqdm(range(0, epochs)):
            out = self.multi_task_training(epoch)
        return out


    def multi_task_training(self, epoch):
        self.model.train()

        self.optimizer.zero_grad()
        out = model(self.data.x_dict, self.data.edge_index_dict)
        train_losses = self.get_losses(out, self.y, self.train_mask)
        train_total_loss = sum(train_losses)
        train_total_loss.backward()
        optimizer.step()

        train_accuracies = self.get_accuracies(out, self.y, self.train_mask)

        print(f'Epoch: {epoch+1}')
        for i, train_loss_acc in enumerate(zip(train_losses, train_accuracies)):
            print(f'\t {self.map_labels[i]} \t {round(train_loss_acc[0].detach().item(), 4)} \t{round(train_loss_acc[1].item(), 2) * 100}%')

        if epoch % 5 == 0:
            self.test(out)

        return out

    def test(self, out):
        val_losses = self.get_losses(out, self.y, self.val_mask)
        test_losses = self.get_losses(out, self.y, self.test_mask)

        val_accuracies = self.get_accuracies(out, self.y, self.val_mask)
        test_accuracies = self.get_accuracies(out, self.y, self.test_mask)

        print(f'*\tOn validation')
        for i, val_loss_acc in enumerate(zip(val_losses, val_accuracies)):
            print(f'\t{self.map_labels[i]}\t {round(val_loss_acc[0].detach().item(), 4)} \t {round(val_loss_acc[1].item(), 2) * 100}%')

        print(f'*\tOn test')
        for i, test_loss_acc in enumerate(zip(test_losses, test_accuracies)):
            print(f'\t{self.map_labels[i]}\t {round(test_loss_acc[0].detach().item(), 4)} \t {round(test_loss_acc[1].item(), 2) * 100}%')

In [ ]:
model = MGNN( hidden_channels=16, out_channels=base_data.num_classes, metadata=data.metadata())
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
gcn = ArtGraphGCNBoost(model, data, optimizer)
out = gcn.train(1000)
torch.save(out, "out.pt")